In [10]:
#here we will write our training and testing data
from keras.datasets import cifar10 
from keras.utils import to_categorical
(X_train,Y_train),(X_test,Y_test)=cifar10.load_data()
X_train=X_train/255
X_test=X_test/255
Y_train =to_categorical(Y_train,num_classes=10)

In [11]:
input_shape_for_model=X_train[0,:,:,:].shape

In [12]:
from keras.models import Input,Model
input=Input(shape=input_shape_for_model)

ConvPool-CNN-C

In [13]:
from tensorflow.python.framework.ops import Tensor
from keras.layers import Conv2D,Dropout,Activation,Average,MaxPooling2D,GlobalAveragePooling2D,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
def ConvPool_CNN_C(model_input :Tensor):
  x=Conv2D(96,kernel_size=(3,3),activation='relu',padding='same')(model_input)
  x=Conv2D(96,kernel_size=(3,3),activation='relu',padding='same')(x)
  x=Conv2D(96,kernel_size=(3,3),activation='relu',padding='same')(x)
  x=MaxPooling2D(pool_size=(3,3),strides=2)(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=MaxPooling2D(pool_size=(3,3),strides=2)(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=Conv2D(192,(1,1),activation='relu')(x)
  x=Conv2D(10,(1,1))(x)
  x=GlobalAveragePooling2D()(x)
  x=Activation(activation='softmax')(x)
  my_model=Model(model_input,x,name='ConvPool_CNN_C')
  return my_model

In [14]:
ConvPool_CNN_model=ConvPool_CNN_C(input)
from keras.callbacks import ModelCheckpoint, TensorBoard

In [15]:
def Train_Model(model,num_epochs=25):
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
  checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True,
                                                 save_best_only=True, mode='auto', period=1)
  tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
  history = model.fit(x=X_train, y=Y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
  weight_files=glob.glob(os.path.join(os.getcwd(),'weights/*'))
  weight_file=max(weight_files,weight_files)
  return weight_file

In [16]:
import glob
import os
weight_file=Train_Model(ConvPool_CNN_model)

Epoch 1/25
1250/1250 [==============================] - 20s 16ms/step - loss: 1.9279 - accuracy: 0.2575 - val_loss: 1.6418 - val_accuracy: 0.3891
Epoch 2/25
1250/1250 [==============================] - 21s 16ms/step - loss: 1.4482 - accuracy: 0.4627 - val_loss: 1.3462 - val_accuracy: 0.4977
Epoch 3/25
1250/1250 [==============================] - 20s 16ms/step - loss: 1.1785 - accuracy: 0.5755 - val_loss: 1.1351 - val_accuracy: 0.5956
Epoch 4/25
1250/1250 [==============================] - 20s 16ms/step - loss: 0.9822 - accuracy: 0.6512 - val_loss: 0.9852 - val_accuracy: 0.6537
Epoch 5/25
1250/1250 [==============================] - 20s 16ms/step - loss: 0.8492 - accuracy: 0.6984 - val_loss: 0.8926 - val_accuracy: 0.6816
Epoch 6/25
1250/1250 [==============================] - 20s 16ms/step - loss: 0.7380 - accuracy: 0.7401 - val_loss: 0.8301 - val_accuracy: 0.7093
Epoch 7/25
1250/1250 [==============================] - 20s 16ms/step - loss: 0.6513 - accuracy: 0.7705 - val_loss: 0.7367 -

In [17]:
def error(model):
  predict=model.predict(X_test,batch_size=32)
  predict=np.argmax(predict,axis=1)
  predict=np.expand_dims(predict,axis=1)
  error=np.sum(np.not_equal(predict,Y_test))/Y_test.shape[0]
  return error

In [18]:
import numpy as np
try:
  weight_file
except NameError:
  ConvPool_CNN_model.load_weights(weight_file)
error(ConvPool_CNN_model)

0.2414

ALL-CNN-C

In [19]:
def all_cnn_c(model_input:Tensor):
  x=Conv2D(96,kernel_size=(3,3),activation='relu',padding='same')(model_input)
  x=Conv2D(96,kernel_size=(3,3),activation='relu',padding='same')(x)
  x=Conv2D(96,kernel_size=(3,3),activation='relu',padding='same',strides=2)(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same',strides=2)(x)
  x=Conv2D(192,(3,3),activation='relu',padding='same')(x)
  x=Conv2D(192,(1,1),activation='relu')(x)
  x=Conv2D(10,(1,1))(x)
  x=GlobalAveragePooling2D()(x)
  x=Activation(activation='softmax')(x)
  model=Model(model_input,x,name='all_cnn')
  return model

In [20]:
all_cnn_model=all_cnn_c(input)

In [21]:
all_cnn_weight_file=Train_Model(all_cnn_model)

Epoch 1/25
1250/1250 [==============================] - 19s 15ms/step - loss: 1.7921 - accuracy: 0.3133 - val_loss: 1.5435 - val_accuracy: 0.4235
Epoch 2/25
1250/1250 [==============================] - 19s 15ms/step - loss: 1.3658 - accuracy: 0.4982 - val_loss: 1.1920 - val_accuracy: 0.5574
Epoch 3/25
1250/1250 [==============================] - 19s 15ms/step - loss: 1.1047 - accuracy: 0.6026 - val_loss: 1.0454 - val_accuracy: 0.6201
Epoch 4/25
1250/1250 [==============================] - 19s 15ms/step - loss: 0.9207 - accuracy: 0.6739 - val_loss: 0.8648 - val_accuracy: 0.6964
Epoch 5/25
1250/1250 [==============================] - 19s 15ms/step - loss: 0.7805 - accuracy: 0.7271 - val_loss: 0.8569 - val_accuracy: 0.6988
Epoch 6/25
1250/1250 [==============================] - 19s 15ms/step - loss: 0.6675 - accuracy: 0.7665 - val_loss: 0.7919 - val_accuracy: 0.7335
Epoch 7/25
1250/1250 [==============================] - 19s 15ms/step - loss: 0.5649 - accuracy: 0.8015 - val_loss: 0.7266 -

In [22]:
try:
  all_cnn_weight_file
except NameError:
  all_cnn_model.load_weights(all_cnn_weight_file)
error(all_cnn_model)

0.2546

Network in Network CNN

In [23]:
def NIN_CNN(model_input:Tensor):
  x=Conv2D(32,(5,5),activation='relu',padding='valid')(model_input)
  x=Conv2D(32,(1,1),activation='relu')(x)
  x=Conv2D(32,(1,1),activation='relu')(x)
  x=MaxPooling2D(2,2)(x)
  x=Dropout(0.5)(x)
  x=Conv2D(64,(3,3),activation='relu',padding='valid')(x)
  x=Conv2D(64,(1,1),activation='relu')(x)
  x=Conv2D(64,(1,1),activation='relu')(x)
  x=MaxPooling2D(2,2)(x)
  x=Dropout(0.5)(x)
  x=Conv2D(128,(3,3),activation='relu',padding='valid')(x)
  x=Conv2D(32,(1,1),activation='relu')(x)
  x=Conv2D(10,(1,1),activation='relu')(x)
  x=GlobalAveragePooling2D()(x)
  x=Activation(activation='softmax')(x)
  model=Model(model_input,x,name='nin_cnn')
  return model


In [24]:
NIN_CNN_Model=NIN_CNN(input)

In [25]:
Nin_cnn_weight_file=Train_Model(NIN_CNN_Model)

Epoch 1/25
1250/1250 [==============================] - 7s 6ms/step - loss: 1.9946 - accuracy: 0.2604 - val_loss: 1.7891 - val_accuracy: 0.3480
Epoch 2/25
1250/1250 [==============================] - 7s 5ms/step - loss: 1.6158 - accuracy: 0.4062 - val_loss: 1.4875 - val_accuracy: 0.4547
Epoch 3/25
1250/1250 [==============================] - 7s 5ms/step - loss: 1.4860 - accuracy: 0.4577 - val_loss: 1.3730 - val_accuracy: 0.5076
Epoch 4/25
1250/1250 [==============================] - 7s 5ms/step - loss: 1.3957 - accuracy: 0.4961 - val_loss: 1.3425 - val_accuracy: 0.5157
Epoch 5/25
1250/1250 [==============================] - 7s 5ms/step - loss: 1.3315 - accuracy: 0.5196 - val_loss: 1.2348 - val_accuracy: 0.5667
Epoch 6/25
1250/1250 [==============================] - 7s 5ms/step - loss: 1.2840 - accuracy: 0.5399 - val_loss: 1.2658 - val_accuracy: 0.5443
Epoch 7/25
1250/1250 [==============================] - 7s 6ms/step - loss: 1.2445 - accuracy: 0.5546 - val_loss: 1.1808 - val_accuracy:

In [26]:
try:
  Nin_cnn_weight_file
except NameError:
  NIN_CNN_Model.load_weights(Nin_cnn_weight_file)
error(NIN_CNN_Model)

0.3063

Three Model Using Ensemble

In [27]:
Models=[ConvPool_CNN_model,all_cnn_model,NIN_CNN_Model]


In [28]:
from typing import Tuple,List
def ensemble(models:List[Model],input:Tensor):
  outputs=[model.outputs[0] for model in models]
  y=Average()(outputs)
  model=Model(input,y,name='ensemble_model')
  return model

In [29]:
ensemble_Model=ensemble(Models,input)

In [30]:
error(ensemble_Model)#the error is lowest than individual models

0.1979